In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip install flask-ngrok

In [5]:
!pip install transformers

In [6]:
#!pip install tensorflow-gpu==1.15

ERROR: Could not find a version that satisfies the requirement tensorflow-gpu==1.15 (from versions: 2.2.0rc1, 2.2.0rc2, 2.2.0rc3, 2.2.0rc4, 2.2.0, 2.2.1, 2.2.2, 2.3.0rc0, 2.3.0rc1, 2.3.0rc2, 2.3.0, 2.3.1, 2.3.2, 2.4.0rc0, 2.4.0rc1, 2.4.0rc2, 2.4.0rc3, 2.4.0rc4, 2.4.0, 2.4.1)
ERROR: No matching distribution found for tensorflow-gpu==1.15


In [1]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [8]:
!pip install pythainlp

In [2]:
from flask import Flask, jsonify, request
from flask_ngrok import run_with_ngrok
from transformers import AutoTokenizer, AutoModel
from transformers import BertTokenizer
from pythainlp.tokenize import word_tokenize
#from google.colab import widgets
import numpy as np
import json

In [3]:
app = Flask(__name__)
run_with_ngrok(app)

In [4]:
pretrained = "monsoon-nlp/bert-base-thai"#@param ["monsoon-nlp/bert-base-thai"] 

In [5]:
#model_path = '/content/drive/MyDrive/LegalBERT/model_finetuning/save_model/1611409445'
model_path = 'C:\\Users\\LENOVO\\Documents\\ProjectBERT\\model_finetuning\\save_model\\1611409445'

In [6]:
session_config = tf.compat.v1.ConfigProto(allow_soft_placement=True)
session_config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=session_config)
tf.compat.v1.saved_model.loader.load(sess, [tf.compat.v1.saved_model.tag_constants.SERVING], model_path)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from C:\Users\LENOVO\Documents\ProjectBERT\model_finetuning\save_model\1611409445\variables\variables


meta_info_def {
  stripped_op_list {
    op {
      name: "Add"
      input_arg {
        name: "x"
        type_attr: "T"
      }
      input_arg {
        name: "y"
        type_attr: "T"
      }
      output_arg {
        name: "z"
        type_attr: "T"
      }
      attr {
        name: "T"
        type: "type"
        allowed_values {
          list {
            type: DT_BFLOAT16
            type: DT_HALF
            type: DT_FLOAT
            type: DT_DOUBLE
            type: DT_UINT8
            type: DT_INT8
            type: DT_INT16
            type: DT_INT32
            type: DT_INT64
            type: DT_COMPLEX64
            type: DT_COMPLEX128
            type: DT_STRING
          }
        }
      }
    }
    op {
      name: "AddV2"
      input_arg {
        name: "x"
        type_attr: "T"
      }
      input_arg {
        name: "y"
        type_attr: "T"
      }
      output_arg {
        name: "z"
        type_attr: "T"
      }
      attr {
        name: "T"
      

In [7]:
name = pretrained
#tokenizer = AutoTokenizer.from_pretrained(name)
#model = AutoModel.from_pretrained(name)
tz = BertTokenizer.from_pretrained(name)

In [8]:
def predict_text(input_value):
    #ตัดคำ
    e=word_tokenize(input_value,engine='newmm') 

    #Extrac Feature input for Prediction
    #tz = BertTokenizer.from_pretrained(name)
    # The senetence to be encoded
    # Encode the sentence
    encoded = tz.encode_plus(
        text=e,  # the sentence to be encoded
        add_special_tokens=True,  # Add [CLS] and [SEP]
        max_length = 128,  # maximum length of a sentence
        pad_to_max_length=True,  # Add [PAD]s
        return_attention_mask = True,  # Generate the attention mask
        return_tensors = 'pt',  # ask the function to return PyTorch tensors
)
        # Get the input IDs and attention mask in tensor format
    input_ids = encoded['input_ids']
    input_ids_new = np.array((input_ids),dtype = 'int64')

    softmax_tensor = sess.graph.get_tensor_by_name('loss/Softmax:0')
    predictions = sess.run(softmax_tensor, {'Placeholder:0': input_ids_new})

    labels = np.array([['Violation', 'family', 'labor', 'contract', 'criminal']])
    for i in range(predictions.shape[1]):
        score = predictions.max()
        if predictions[0,i] == score :
            tag = labels[0,i]
            tag = str(tag)
    
    print(predictions)
    print((score*100))
    return(tag)

In [11]:
predict_text("ลูกยืมเงินไปซื้อยาเสพติดแล้วไม่คืน")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\LENOVO\miniconda3\lib\site-packages\transformers\tokenization_utils_base.py:2137: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


[[1.7631967e-03 7.2315813e-04 3.5477843e-04 9.8759580e-01 9.5631639e-03]]
98.7595796585083


'contract'

In [9]:
@app.route("/predict", methods=["POST"])
def predict():
  result = 0
  if request.method == "POST":    		
    input_value = request.form["text"]
    tag_law = predict_text(input_value)
    # ประมวลผล #
    print(input_value)
    print(tag_law)

    json_data = json.dumps({'result':tag_law})

    return json_data

In [ ]:
if __name__ == '__main__':
    app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://590f58b6d4d8.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
